In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
import matplotlib
matplotlib.rcParams["animation.ffmpeg_path"] = "/usr/bin/ffmpeg"


# Load dataset
df = pd.read_csv("/kaggle/input/datasets/adityaanilkapile/drone-swarm-optimisation/synthetic_drone_swarm_dataset.csv")

print("Dataset loaded:", df.shape)
display(df.head())


Dataset loaded: (20000, 11)


,drone_id,timestamp,x,y,z,velocity,battery_level,signal_strength,collision_warning,formation_pattern,mission_status
0,0,0,2.483571,-0.691322,3.238443,2.340280,99.220027,52.904181,0,random,return
1,1,0,7.615149,-2.904391,-2.625849,2.751068,98.478789,76.237822,0,line,delivery
2,2,0,-5.064156,1.571237,-4.540120,6.841050,96.074120,59.983689,0,random,exploration
3,3,0,-7.061519,0.611096,-2.577178,2.557862,99.674742,97.444277,0,circle,surveillance
4,4,0,-3.003193,-1.458469,-3.008533,1.830574,97.524115,51.719426,0,random,surveillance


In [2]:
import sys
import os
sys.path.append(os.path.abspath("src"))


In [3]:
os.makedirs("src/visualization", exist_ok=True)
os.makedirs("outputs", exist_ok=True)


In [4]:
df.columns

Index(['drone_id', 'timestamp', 'x', 'y', 'z', 'velocity', 'battery_level',
       'signal_strength', 'collision_warning', 'formation_pattern',
       'mission_status'],
      dtype='object')

In [5]:
# ENVIRONMENT BOUNDS
bounds = np.array([
    [0, 50],   # x
    [0, 50],   # y
    [0, 20]    # z
])

#OBSTACLES 
np.random.seed(42)
N_OBS = 6
obstacles = [
    (
        np.random.uniform(bounds[0,0], bounds[0,1]),
        np.random.uniform(bounds[1,0], bounds[1,1]),
        np.random.uniform(3, 6)
    )
    for _ in range(N_OBS)
]

In [6]:
# DRONE PHYSICS
MAX_SPEED = 3.0
MAX_ACCEL = 1.0
DT = 1.0

def simulate_step(pos, vel, target):
    desired = target - pos
    dist = np.linalg.norm(desired, axis=1, keepdims=True) + 1e-6
    direction = desired / dist

    accel = direction * MAX_ACCEL
    vel = vel + accel * DT

    speed = np.linalg.norm(vel, axis=1, keepdims=True)
    vel = vel / np.maximum(1, speed / MAX_SPEED)

    pos = pos + vel * DT
    return pos, vel

In [7]:
# OBSTACLE PENALTY

def obstacle_penalty(positions):
    pen = 0
    for ox, oy, r in obstacles:
        d = np.sqrt((positions[:,0]-ox)**2 + (positions[:,1]-oy)**2)
        overlap = np.maximum(0, r - d)
        pen += np.sum(overlap)
    return pen

In [8]:
# TARGET FORMATION
def circle_formation(N, radius=10):
    angles = np.linspace(0, 2*np.pi, N, endpoint=False)
    x = 25 + radius * np.cos(angles)
    y = 25 + radius * np.sin(angles)
    z = np.full(N, 10)
    return np.vstack([x,y,z]).T

N_DRONES = 8
target_positions = circle_formation(N_DRONES)

In [9]:
# FITNESS FUNCTION
def fitness(positions, target_positions, safe_dist=2.0):
    N = len(positions)

    form_err = np.mean(np.linalg.norm(positions - target_positions, axis=1))

    collision_pen = 0
    for i in range(N):
        for j in range(i+1, N):
            d = np.linalg.norm(positions[i] - positions[j])
            if d < safe_dist:
                collision_pen += (safe_dist - d)

    obs_pen = obstacle_penalty(positions)

    energy = np.mean(np.linalg.norm(positions, axis=1))

    return form_err + 6*collision_pen + 8*obs_pen + 0.1*energy

In [10]:
# PSO CLASS

class PSO:
    def __init__(self, n_particles, N, bounds):
        self.np = n_particles
        self.N = N
        self.bounds = bounds

        self.pos = np.random.uniform(
            bounds[:,0], bounds[:,1],
            (n_particles, N, 3)
        )
        self.vel = np.zeros((n_particles, N, 3))

        self.pbest = self.pos.copy()
        self.pbest_val = np.full(n_particles, np.inf)

        self.gbest = None
        self.gbest_val = np.inf

    def step(self, w=0.9, c1=0.8, c2=0.8):
        r1 = np.random.rand(self.np, self.N, 3)
        r2 = np.random.rand(self.np, self.N, 3)

        cognitive = c1 * r1 * (self.pbest - self.pos)
        social    = c2 * r2 * (self.gbest - self.pos)

        accel = w*self.vel + cognitive + social

        accel_mag = np.linalg.norm(accel, axis=2, keepdims=True)
        accel = accel / np.maximum(1, accel_mag / MAX_ACCEL)

        # inertia + acceleration
        self.vel = 0.9 * self.vel + accel

        # small exploration noise (prevents early lock)
        self.vel += 0.03 * np.random.randn(*self.vel.shape)

        # speed limiting
        speed = np.linalg.norm(self.vel, axis=2, keepdims=True)
        self.vel = self.vel / np.maximum(1, speed / MAX_SPEED)


        self.pos += self.vel * DT

        self.pos[:,:,0] = np.clip(self.pos[:,:,0], bounds[0,0], bounds[0,1])
        self.pos[:,:,1] = np.clip(self.pos[:,:,1], bounds[1,0], bounds[1,1])
        self.pos[:,:,2] = np.clip(self.pos[:,:,2], bounds[2,0], bounds[2,1])

        vals = np.array([
            fitness(self.pos[i], target_positions)
            for i in range(self.np)
        ])

        mask = vals < self.pbest_val
        self.pbest[mask] = self.pos[mask]
        self.pbest_val[mask] = vals[mask]

        g_idx = np.argmin(self.pbest_val)
        if self.pbest_val[g_idx] < self.gbest_val:
            self.gbest = self.pbest[g_idx].copy()
            self.gbest_val = self.pbest_val[g_idx]


In [11]:
# RUN PSO (single optimisation run)
pso = PSO(n_particles=40, N=N_DRONES, bounds=bounds)

# initialise global best from initial swarm
pso.gbest = pso.pos[0].copy()

ITERATIONS = 150

history = []        # all particles
gbest_history = []  # best swarm

for _ in range(ITERATIONS):
    pso.step()

    history.append(pso.pos.copy())
    gbest_history.append(pso.gbest.copy())

print("Best fitness:", pso.gbest_val)


Best fitness: 9.276632135305587


In [12]:
code = """
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
import matplotlib.lines as mlines

def render_best_swarm(history, targets, obstacles=None,
                      out_name="outputs/best_swarm.mp4"):

    history = np.asarray(history)
    targets = np.asarray(targets)[:, :2]

    W = 50
    DRONE_SIZE = 120
    TARGET_SIZE = 140

    fig, ax = plt.subplots(figsize=(8,8), dpi=120)

    ax.set_xlim(0, W)
    ax.set_ylim(0, W)
    ax.set_aspect("equal")
    ax.set_xticks([])
    ax.set_yticks([])

    ax.set_title("Best Swarm Convergence", fontsize=14, pad=12)

    # HUD
    hud = ax.text(
        0.02, 0.98, "",
        transform=ax.transAxes,
        ha="left", va="top",
        fontsize=11,
        bbox=dict(facecolor="white", alpha=0.9, boxstyle="round")
    )

    # Symbol index legend
    drone_handle = mlines.Line2D([], [], color="tab:blue", marker="o",
                                 linestyle="None", markersize=8,
                                 label="Drone")

    target_handle = mlines.Line2D([], [], color="red", marker="X",
                                  linestyle="None", markersize=8,
                                  label="Target")

    obstacle_handle = mlines.Line2D([], [], color="black", marker="o",
                                    linestyle="None", markersize=8,
                                    markerfacecolor="none",
                                    label="Obstacle")

    ax.legend(
        handles=[drone_handle, target_handle, obstacle_handle],
        loc="upper right",
        framealpha=0.95,
        title="Symbol Index"
    )

    # Initial swarm
    p0 = history[0][:, :2]

    drone_scatter = ax.scatter(
        p0[:,0], p0[:,1],
        s=DRONE_SIZE,
        color="tab:blue",
        edgecolors="black",
        zorder=5
    )

    target_scatter = ax.scatter(
        targets[:,0], targets[:,1],
        s=TARGET_SIZE,
        marker="X",
        color="red",
        zorder=6
    )

    # obstacles
    if obstacles is not None:
        for ox, oy, r in obstacles:
            circ = plt.Circle((ox, oy), r,
                              fill=False,
                              edgecolor="black",
                              linewidth=2)
            ax.add_patch(circ)

    def draw(i):
        pos = history[i][:, :2]
        drone_scatter.set_offsets(pos)

        hud.set_text(
            f"Iteration: {i+1}/{len(history)}\\n"
            f"Drones: {len(pos)}"
        )

        return drone_scatter, hud

    anim = FuncAnimation(
        fig, draw,
        frames=len(history),
        interval=40,
        blit=True
    )

    writer = FFMpegWriter(fps=10)
    anim.save(out_name, writer=writer)
    plt.close(fig)

    print("Saved:", out_name)
"""
with open("/kaggle/working/src/visualization/render_best_swarm.py","w") as f:
    f.write(code)


In [13]:
from visualization.render_best_swarm import render_best_swarm

render_best_swarm(
    gbest_history,
    target_positions,
    obstacles
)


Saved: outputs/best_swarm.mp4


In [14]:
from IPython.display import Video

Video("outputs/best_swarm.mp4", embed=True)


In [15]:
import os, shutil

base = "/kaggle/working/drone_swarm_optimisation"

# create folders
os.makedirs(f"{base}/src/visualization", exist_ok=True)
os.makedirs(f"{base}/outputs", exist_ok=True)

# copy renderer
shutil.copy(
    "/kaggle/working/src/visualization/render_best_swarm.py",
    f"{base}/src/visualization/render_best_swarm.py"
)

# copy video
shutil.copy(
    "/kaggle/working/outputs/best_swarm.mp4",
    f"{base}/outputs/best_swarm.mp4"
)

# README
readme = """# Drone Swarm Optimisation using PSO

This project demonstrates particle swarm optimisation (PSO) for
multi-drone formation control with obstacle avoidance.

## Features
- 3D drone swarm simulation
- PSO formation optimisation
- Circular target formation
- Obstacle avoidance
- Convergence animation

## Structure
src/visualization/render_best_swarm.py  → animation renderer  
outputs/best_swarm.mp4                  → convergence video  
notebook.ipynb                          → PSO simulation  

## Usage
Run notebook.ipynb to reproduce results.
"""

with open(f"{base}/README.md", "w") as f:
    f.write(readme)

# zip project
shutil.make_archive(
    "/kaggle/working/drone_swarm_optimisation",
    "zip",
    base
)

print("Swarm project exported")


Swarm project exported


In [16]:
from IPython.display import FileLink

FileLink("drone_swarm_optimisation.zip")


/kaggle/working/drone_swarm_optimisation.zip